# searchMemberByPhone

In [ ]:
import json
from datetime import datetime

members = {
    "member_details": 
 [
      {"name": "Siddhesh", "phone_number": '+1209938588', 
       "member_number": "12345678", "dob": datetime.strptime('1-11-1990', '%m-%d-%Y').date(), 
       'address':'xyz abc', 'zip': "17207", 'possible_intent': 'order status'},
        
      {"name": "Pramod", "phone_number": '+12032539043', 
       "member_number": "12348765", "dob": datetime.strptime('2-12-1991', '%m-%d-%Y').date(), 
       'address':'xyz def', 'zip': "17201", 'possible_intent': 'order cancellation'},
        
      {"name": "Akhil", "phone_number": '+12019338234', 
       "member_number": "87651234", "dob": datetime.strptime('3-13-1992', '%m-%d-%Y').date(), 
       'address':'xyz ghi', 'zip': "17202", 'possible_intent': 'order updation'},
        
      {"name": "Arun", "phone_number": '+12019338553', 
       "member_number": "43215678", "dob": datetime.strptime('4-14-1993', '%m-%d-%Y').date(), 
       'address':'xyz jkl', 'zip': "17203", 'possible_intent': 'order payment'}      
 ]
}

def name(id):
    return members['member_details'][id]['name']

def member_number(id):
    return members['member_details'][id]['member_number']

def zip_code(id):
    return members['member_details'][id]['zip']

def predictiveIntent(id):
    return members['member_details'][id]['possible_intent']
    
def lambda_handler(event, context):
    phone_numbers = [members['member_details'][i]['phone_number'] for i in range(len(members['member_details']))]
    customer_number = event['Details']['Parameters']['customer_number']
    flag = False

    if customer_number in phone_numbers:
        flag = True
        index = phone_numbers.index(customer_number)
        intent = predictiveIntent(index)
        customer_name = name(index)
        customer_member_number = member_number(index)
        zipcode = zip_code(index)
        
        resultMap = {
        'flag':flag,
        'intent':intent,
        'customer_name':customer_name,
        'zipcode':zipcode,
        'customer_member_number':customer_member_number,
        'customer_number':customer_number
        }
        return resultMap
    else:
        resultMap = {
        'flag':flag,
        }
        return resultMap 


# searchMemberByMiscellaneous

In [ ]:
import json
from datetime import datetime

members = {
    "member_details": 
 [
      {"name": "Siddhesh", "phone_number": '+1209938588', 
       "member_number": "12345678", "dob": datetime.strptime('1-11-1990', '%m-%d-%Y').date(), 
       'address':'xyz abc', 'zip': "17207", 'possible_intent': 'order status'},
        
      {"name": "Pramod", "phone_number": '+12032539043', 
       "member_number": "12348765", "dob": datetime.strptime('2-12-1991', '%m-%d-%Y').date(), 
       'address':'xyz def', 'zip': "17201", 'possible_intent': 'order cancellation'},
        
      {"name": "Akhil", "phone_number": '+12019338234', 
       "member_number": "87651234", "dob": datetime.strptime('3-13-1992', '%m-%d-%Y').date(), 
       'address':'xyz ghi', 'zip': "17202", 'possible_intent': 'order updation'},
        
      {"name": "Arun", "phone_number": '+12019338553', 
       "member_number": "43215678", "dob": datetime.strptime('4-14-1993', '%m-%d-%Y').date(), 
       'address':'xyz jkl', 'zip': "17203", 'possible_intent': 'order payment'}      
 ]
}

def name(id):
    return members['member_details'][id]['name']

def member_number(id):
    return members['member_details'][id]['member_number']

def zip_code(id):
    return members['member_details'][id]['zip']

def predictiveIntent(id):
    return members['member_details'][id]['possible_intent']
    
def lambda_handler(event, context):
    # values from flow
    customer_dob_year = event['Details']['Parameters']['customer_dob']
    customer_zip = event['Details']['Parameters']['customer_zip']
    customer_member_number = event['Details']['Parameters']['customer_member_number']
    
    # lists of values
    dates = [members['member_details'][i]['dob'] for i in range(len(members['member_details']))]
    years = [str(i.year) for i in dates]
    zip_codes = [members['member_details'][i]['zip'] for i in range(len(members['member_details']))]
    id_numbers = [members['member_details'][i]['member_number'] for i in range(len(members['member_details']))]
    
    flag = False

    if (customer_dob_year in years) and (customer_zip in zip_codes) and (customer_member_number in id_numbers):
        flag = True
        index = years.index(customer_dob_year)
        intent = predictiveIntent(index)
        customer_name = name(index)

        resultMap = {
        'flag':flag,
        'intent':intent,
        'customer_name':customer_name,
        }
        return resultMap
    else:
        resultMap = {
        'flag':flag,
        }
        return resultMap 


# gluejob

In [ ]:
import sys
from pyspark import SparkContext
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame

# Create a Glue context and Spark session
glue_context = GlueContext(SparkContext.getOrCreate())
spark = glue_context.spark_session

# Read the input data from S3
input_dyf = glue_context.create_dynamic_frame.from_options(
    connection_type='s3',
    connection_options={'paths': ["s3://athenaoutputfromquery/Unsaved/2023/02/18/5c439513-e2c1-48b9-bd3f-1e8e1d710eb2.csv"]},
    format='csv',
    format_options={'withHeader': True}
)

data_frame = input_dyf.toDF()
df = data_frame.select("attributes")


import numpy as np
import pandas as pd

def stringtodict(str_data):
    pairs = str_data[1:-1].split(", ")

    # Split each key-value pair into key and value
    data = [pair.split("=") for pair in pairs]

    # Create dictionary from key-value pairs
    dict_data = {key: value if value != "null" else None for key, value in data}

    # Print the resulting dictionary
    return dict_data

rows_list = df.select("*").collect()
rows_array = np.array([stringtodict(row[0]) for row in rows_list])

# Print the resulting numpy array
# print(rows_array)

customer_choice_data = [int(rows_array[i]['customer_choice']) for i in range(len(rows_array))]
customer_number_data = [rows_array[i]['customer_number'] for i in range(len(rows_array))]
customer_dob_data = [rows_array[i]['customer_dob'] for i in range(len(rows_array))]
customer_member_number_data = [rows_array[i]['customer_member_number'] for i in range(len(rows_array))]
customer_zip_data = [rows_array[i]['customer_zip'] for i in range(len(rows_array))]

# Create dataframe
df = pd.DataFrame({"customer_choice": customer_choice_data, "customer_number": customer_number_data, "customer_dob": customer_dob_data, "customer_member_number": customer_member_number_data, "customer_zip": customer_zip_data})

df.to_csv('s3://gluetransformationoutput/transformed_data.csv', index=False)



# contact trace record json data

In [ ]:
{
  "AWSAccountId": "232154061460",
  "AWSContactTraceRecordFormatVersion": "2017-03-10",
  "Agent": null,
  "AgentConnectionAttempts": 0,
  "AnsweringMachineDetectionStatus": null,
  "Attributes": {
    "customer_choice": "3",
    "customer_dob": "1990",
    "customer_member_number": "12345678",
    "customer_number": "+12019938588",
    "customer_zip": "17207"
  },
  "Campaign": {
    "CampaignId": null
  },
  "Channel": "VOICE",
  "ConnectedToSystemTimestamp": "2023-02-19T03:12:06Z",
  "ContactDetails": {},
  "ContactId": "6e117a92-ccaa-4e48-acba-ccac045ba574",
  "CustomerEndpoint": {
    "Address": "+12019938588",
    "Type": "TELEPHONE_NUMBER"
  },
  "DisconnectReason": "CONTACT_FLOW_DISCONNECT",
  "DisconnectTimestamp": "2023-02-19T03:12:46Z",
  "InitialContactId": null,
  "InitiationMethod": "INBOUND",
  "InitiationTimestamp": "2023-02-19T03:12:06Z",
  "InstanceARN": "arn:aws:connect:us-west-2:232154061460:instance/4afffe99-5005-4677-a2e4-a5c4c791e6bf",
  "LastUpdateTimestamp": "2023-02-19T03:13:51Z",
  "MediaStreams": [
    {
      "Type": "AUDIO"
    }
  ],
  "NextContactId": null,
  "PreviousContactId": null,
  "Queue": null,
  "Recording": null,
  "Recordings": null,
  "References": [],
  "ScheduledTimestamp": null,
  "SystemEndpoint": {
    "Address": "+14155236336",
    "Type": "TELEPHONE_NUMBER"
  },
  "Tags": {},
  "TransferCompletedTimestamp": null,
  "TransferredToEndpoint": null,
  "VoiceIdResult": null
}